# Sim III: Contact dynamics

In [ ]:
from utils import magic_donotload

import numpy as np
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import meshcat
import matplotlib.pyplot as plt

from utils.contact_dyn import create_cubes, computeContactProblem
from utils.visualization import sub_sample

## I - Implementation of Projected Gauss Seidel

We are going to implement a Bullet-like simulator.
You should implement a contact solver using PGS (cf. the course slides on Bullet) with the following API:

In [ ]:
#%load -r 3-25 utils/pgs.py
def solve_contact(G: np.ndarray,g: np.ndarray, mus: list, tol : float = 1e-6, max_iter :int = 100) -> np.ndarray:
    """PGS algorithm solving a contact problem with frictions.

    Args:
        G (np.ndarray): Delassus matrix.
        g (np.ndarray): free velocity of contact points.
        mus (list): list of coefficients of friction for the contact points.
        tol (float, optional): solver tolerance. Defaults to 1e-6.
        max_iter (int, optional): maximum number of iterations for the solver. Defaults to 100.

    Returns:
        np.ndarray: contact impulses.
    """
    # TODO : PGS
    return 

### Simulating a cube on a plane

We build the pinocchio model of a cube and a plane (which stands for the floor) and we fix the simulation parameters (time step, contact solver accuracy etc):

In [ ]:
np.random.seed(1234)
pin.seed(12345)

## First example: a cube falling on the floor

cube_dimension = 0.2  # size of cube
cube_mass = 1.0  # mass of cube
mu = 0.95  # friction parameter
eps = 0.0  # elasticity
model, geom_model, visual_model, data, geom_data, visual_data, actuation = create_cubes(
    [cube_dimension], [cube_mass], mu, eps
) # creating pinocchio models and datas


duration = 1. # duration of simulation
dt = 1e-3 # time step duration
T = int(duration/dt) # number of time steps

tolerance = 1e-6 #contact solver accuracy
max_iter = 100 #maximum number of iterations of the contact solver

Simulation is runned for T time steps.

In [ ]:
# initial state
q0 = model.qinit.copy()
v0 = np.zeros(model.nv)
q0[2] = cube_dimension
rand_place = pin.SE3.Random()
q0[-4:] = pin.SE3ToXYZQUAT(rand_place)[-4:]

q, v = q0.copy(), v0.copy()

qs, vs = [q0], [v0] #arrays to store trajectory

for t in range(T): # simulation loop 
    tau = np.zeros(model.nv)
    pin.updateGeometryPlacements(model, data, geom_model, geom_data, q)
    pin.computeCollisions(geom_model, geom_data, False)
    J, vf, Del,g, mus = computeContactProblem(model, data, geom_model, geom_data, q, v, tau, dt)
    if J is not None:
        lam = solve_contact(Del, g, mus, tolerance, max_iter)
        dv = dt*pin.aba(model, data, q, v, tau + J.T @ lam/dt)
        v += dv
    else:
        v = vf
    q = pin.integrate(model , q, v*dt)
    qs += [q]
    vs += [v]

### Visualizing

We visualize the simulated trajectory inside the Meshcat visualizer:

In [ ]:
vizer = MeshcatVisualizer(model, geom_model, visual_model)
vizer.initViewer(open=False, loadModel=True)

vizer.viewer["plane"].set_object(meshcat.geometry.Box(np.array([20, 20, 0.1])))
placement = np.eye(4)
placement[:3, 3] = np.array([0, 0, -0.05])
vizer.viewer["plane"].set_transform(placement)
vizer.display(q0)

cp1 = [0.8, 0.0, 0.2] #camera position
cps_ = [cp1]
numrep = len(cps_)
rps_ = [np.zeros(3)]*numrep

max_fps = 30.
fps = min([max_fps,1./dt])
qs = sub_sample(qs,dt*T, fps)
vs = sub_sample(vs,dt*T, fps)

def get_callback(i: int):
    def _callback(t):
        pin.forwardKinematics(model, vizer.data, qs[t], vs[t])
    return _callback


In [ ]:
vizer.viewer.jupyter_cell()

In [ ]:
for i in range(numrep):
    vizer.play(
        qs,
        1./fps,
        get_callback(i)
    )

## II - Impact of the parameters of the solver

You should implement PGS with over-relaxation and compute the value of the Signorini complementarity at each iteration. Adding over-relaxation to the PGS algorithm only slightly modifies the original algorithm: the step of the original PGS is scaled by the over-relaxation parameter, $\alpha_{or}$.
The goal here is to observe the effects of the choice of the over-relaxation parameter on the simulator performance. In particular, we will inspect the Signorini complementarity to monitor the convergence of the simulator.

In [ ]:
#%load -r 29-60 utils/pgs.py
def solve_contact_over_relax(G: np.ndarray,g: np.ndarray, mus: list, dt: float, tol : float = 1e-6, max_iter :int = 100, alpha_or : float = 1.) -> (np.ndarray, np.ndarray):
    """PGS algorithm solving a contact problem with frictions.

    Args:
        G (np.ndarray): Delassus matrix.
        g (np.ndarray): free velocity of contact points.
        mus (list): list of coefficients of friction for the contact points.
        dt (float): time step.
        tol (float, optional): solver tolerance. Defaults to 1e-6.
        max_iter (int, optional): maximum number of iterations for the solver. Defaults to 100.
        alpha_or (float, optional): over-relaxation parameter. Defaults to 1.

    Returns:
        np.ndarray: contact impulses.
        np.ndarray: value of Signorini complementarity accross iterations of the algorithm.
    """
    # TODO
    return

In [ ]:
q0 = model.qinit.copy()
v0 = np.zeros(model.nv)
q, v = q0.copy(), v0.copy()
tau = np.zeros(model.nv)
pin.updateGeometryPlacements(model, data, geom_model, geom_data, q)
pin.computeCollisions(geom_model, geom_data, False)
J, vf, Del,g, mus = computeContactProblem(model, data, geom_model, geom_data, q, v, tau, dt)

In [ ]:
max_iter = 50
iterations = [i for i in range(max_iter)]
alpha_ors = [.6,1.,1.6]
plt.figure()
for alpha_or in alpha_ors:
    lam, sig_comps = solve_contact_over_relax(Del, g, mus, dt, tolerance, max_iter, alpha_or=alpha_or)
    plt.plot(iterations, sig_comps, label = r"$\alpha_{or}=$"+str(alpha_or))
plt.legend()
plt.xlabel("Iterations", fontsize=20)
plt.ylabel("Signorini complementarity", fontsize=20)
plt.show()

## III - Instability of PGS

### Simulating two stacked cubes

We define the pinocchio model of 2 cubes of very different mass (1g vs 1e3kg) with a plane (standing for the floor). The ill-conditionning of the problem should hinder the convergence of PGS and cause the simulation to fail.

In [ ]:
cube_dimension = 0.2  # size of cube
cube1_mass = 1e-3  # mass of cube 1
cube2_mass = 1e3  # mass of cube 2
mu = 0.9  # friction parameter between cube and floor
el = 0.
comp = 0.
model, geom_model, visual_model, data, geom_data, visual_data, actuation = create_cubes(
    [cube_dimension, cube_dimension], [cube1_mass, cube2_mass], mu, el
)

# Number of time steps
T = 100
dt = 1e-3

# Physical parameters of the contact problem
Kb = 1e-4*0.  # Baumgarte
eps = 0.0  # elasticity

In [ ]:
# initial state
q0 = pin.neutral(model)
q0[2] = cube_dimension / 2 + cube_dimension/50.
q0[9] = 3. * cube_dimension / 2 + 3*cube_dimension/50.
v0 = np.zeros(model.nv)
q, v = q0.copy(), v0.copy()

qs, vs = [q0], [v0] #arrays to store trajectory

for t in range(T): # simulation loop 
    tau = np.zeros(model.nv)
    pin.updateGeometryPlacements(model, data, geom_model, geom_data, q)
    pin.computeCollisions(geom_model, geom_data, False)
    J, vf, Del,g, mus = computeContactProblem(model, data, geom_model, geom_data, q, v, tau, dt)
    if J is not None:
        lam, _ = solve_contact_over_relax(Del, g, mus, dt, tolerance, max_iter)
        dv = dt*pin.aba(model, data, q, v, tau + J.T @ lam/dt)
        v += dv
    else:
        v = vf
    q = pin.integrate(model , q, v*dt)
    qs += [q]
    vs += [v]

### Visualization

We visualize the trajectory of the 2 stacked cubes:

In [ ]:
vizer = MeshcatVisualizer(model, geom_model, visual_model)
vizer.initViewer(open=False, loadModel=True)

vizer.viewer["plane"].set_object(meshcat.geometry.Box(np.array([20, 20, 0.1])))
placement = np.eye(4)
placement[:3, 3] = np.array([0, 0, -0.05])
vizer.viewer["plane"].set_transform(placement)
vizer.display(q0)

cp1 = [0.8, 0.0, 0.2] #camera position
cps_ = [cp1]
numrep = len(cps_)
rps_ = [np.zeros(3)]*numrep

max_fps = 30.
fps = min([max_fps,1./dt])
qs = sub_sample(qs,dt*T, fps)
vs = sub_sample(vs,dt*T, fps)

def get_callback(i: int):
    def _callback(t):
        pin.forwardKinematics(model, vizer.data, qs[t], vs[t])
    return _callback



In [ ]:
vizer.viewer.jupyter_cell()

In [ ]:
for i in range(numrep):
    vizer.play(
        qs,
        1./fps,
        get_callback(i)
    )

### Inspecting convergence

We monitor the evolution of the Signorini complementarity across the iterations of PGS during the simulation of the first time-step:

In [ ]:
# initial state
q0 = pin.neutral(model)
q0[2] = cube_dimension / 2 + cube_dimension/50.
q0[9] = 3. * cube_dimension / 2 + 3*cube_dimension/50.
v0 = np.zeros(model.nv)
q, v = q0.copy(), v0.copy()

tau = np.zeros(model.nv)
pin.updateGeometryPlacements(model, data, geom_model, geom_data, q)
pin.computeCollisions(geom_model, geom_data, False)
J, vf, Del,g, mus = computeContactProblem(model, data, geom_model, geom_data, q, v, tau, dt)
lam, sig_comps  = solve_contact_over_relax(Del, g, mus, dt, tolerance, max_iter)

We will inspect the eigenvalues in order to have an idea of the conditionning of the problem:

In [ ]:
print("eigenvalues:", np.linalg.eigvalsh(Del))

In [ ]:
plt.figure()
iterations = [i for i in range(max_iter)]
plt.plot(iterations, sig_comps)
plt.xlabel("Iterations", fontsize=20)
plt.ylabel("Signorini complementarity", fontsize=20)
plt.show()

## IV - Internal forces

### Simulating a dragged cube

We create the pinocchio model of a cube on a plane:

In [ ]:
cube_dimension = 0.2  # size of cube
cube_mass = 1.0  # mass of cube
mu = 0.95  # friction parameter
eps = 0.0  # elasticity
model, geom_model, visual_model, data, geom_data, visual_data, actuation = create_cubes(
    [cube_dimension], [cube_mass], mu, eps
)

# duration of simulation
duration = .5
# time steps
dt = 1e-3
T = int(duration/dt)

# numerical precision
tolerance = 1e-6
max_iter = 100

We drag the cube on a plane with an increasing force along the y-axis and observe the direction of the friction forces at each contact point:

In [ ]:
q0 = model.qinit.copy()
v0 = np.zeros(model.nv)

q, v = q0.copy(), v0.copy()

qs, vs = [q0], [v0] #arrays to store trajectory

internal_forces = np.zeros((T,4)) # store contact force along x axis for each contact point at every time step

for t in range(T): # simulation loop 
    tau = np.zeros(model.nv)
    tau[1] = t*.1 # applying an increasing force pushing the cube along y axis
    pin.updateGeometryPlacements(model, data, geom_model, geom_data, q)
    pin.computeCollisions(geom_model, geom_data, False)
    J, vf, Del,g, mus = computeContactProblem(model, data, geom_model, geom_data, q, v, tau, dt)
    if J is not None:
        lam = solve_contact(Del, g, mus, tolerance, max_iter)
        dv = dt*pin.aba(model, data, q, v, tau + J.T @ lam/dt)
        v += dv
        for i in range(4):
            internal_forces[t,i] = lam[3*i]/dt
    else:
        v = vf
    q = pin.integrate(model , q, v*dt)
    qs += [q]
    vs += [v]

We visualize the simulated trajectory inside the Meshcat visualizer.

In [ ]:
vizer = MeshcatVisualizer(model, geom_model, visual_model)
vizer.initViewer(open=False, loadModel=True)

vizer.viewer["plane"].set_object(meshcat.geometry.Box(np.array([20, 20, 0.1])))
placement = np.eye(4)
placement[:3, 3] = np.array([0, 0, -0.05])
vizer.viewer["plane"].set_transform(placement)
vizer.display(q0)

cp1 = [1., 0.0, 0.2] #camera position
cps_ = [cp1]
numrep = len(cps_)
rps_ = [np.zeros(3)]*numrep

max_fps = 30.
fps = min([max_fps,1./dt])
q0[2] = cube_dimension
rand_place = pin.SE3.Random()
q0[-4:] = pin.SE3ToXYZQUAT(rand_place)[-4:]
qs = sub_sample(qs,dt*T, fps)
vs = sub_sample(vs,dt*T, fps)

def get_callback(i: int):
    def _callback(t):
        pin.forwardKinematics(model, vizer.data, qs[t], vs[t])
    return _callback


In [ ]:
vizer.viewer.jupyter_cell()

In [ ]:
for i in range(numrep):
    vizer.play(
        qs,
        1./fps,
        get_callback(i)
    )

We plot the friction component along the x-axis:

In [ ]:
plt.figure()
timesteps = [t for t in range(T)]
plt.plot(timesteps, internal_forces)
plt.xlabel("Iterations", fontsize=20)
plt.ylabel("Time-step", fontsize=20)
plt.show()